In [27]:
import pandas as pd
import numpy as np
from khaiii import KhaiiiApi

In [28]:
data = pd.read_csv("./modi_data/kor_full.csv")
data.head()

,no,year,title,kor_full
0,1,1997,멀티미디어 원격교육에 관한 연구,멀티미디어 원격교육에 관한 연구
1,2,1997,교육용 하이퍼미디어 자료 편집기에 관한 연구,교육용 하이퍼미디어 자료 편집기에 관한 연구
2,3,1997,인터넷 기반의 코스웨어의 설계 및 구현,인터넷 기반의 코스웨어의 설계 및 구현
3,4,1997,Web에서의 협력 환경 구축 방안 연구,Web에서의 협력 환경 구축 방안 연구
4,5,1997,열린교육에서의 개별화수업과 CAI,열린교육에서의 개별화수업과 CAI


In [29]:
dfWordList = pd.read_excel("./khaiii_word_cor.xlsx")
dfWordList2 = pd.read_excel("./khaiii_word_cor_etc.xlsx")
#print(dfWordList2)

dfWordDel = dfWordList[dfWordList["수정"] == "삭제"]
dfWordMod = dfWordList[dfWordList["수정"] != "삭제"]
dfWordDiv = dfWordList2
#print(dfWordMod)

seriesDelete = dfWordDel["기존"]
stopword = []
for word in seriesDelete.values:
    stopword.append(word)
#print(stopword)

seriesModify = dfWordMod["기존"]
modiword = []
for word in seriesModify.values:
    modiword.append(word)
#print(len(modiword))

seriesModify2 = dfWordMod["수정"]
modiword2 = []
for word in seriesModify2.values:
    modiword2.append(word)
#print(len(modiword2))

seriesDivide = dfWordDiv["기존"]
divword = []
for word in seriesDivide.values:
    divword.append(word)
#print(divword)
#print(len(divword))

seriesDivide2 = dfWordDiv["수정"]
divword2 = []
for words in seriesDivide2.values:
    divword2.append(words.split(', '))
#print(divword2)
#print(len(divword2))

In [30]:
api = KhaiiiApi()
def khaiiiTokenizer(raw, stopword=stopword, pos=['NNG', 'NNP', 'NNB', 'NP', 'NR', 'SL']): # 일반명사 고유명사 의존명사 대명사 수사 외국어
    list = []
    skip = 0

    for word in api.analyze(raw): #raw data
        #print(word)
        
        for i, morph in enumerate(word.morphs):
            #print(morph.lex)
            if skip == 1: 
                #print(morph.lex) # '지능'
                skip = 0
                continue

            if morph.lex == '인공' and i+1 < len(word.morphs) and word.morphs[i+1].lex == "지능":
                #print(morph.lex + word.morphs[i+1].lex) # 인공지능
                list.append(morph.lex + word.morphs[i+1].lex)
                skip = 1
                continue

            if len(morph.lex) > 1 and morph.tag in pos and morph.lex not in stopword: 
                if morph.tag == 'SL':
                    morph.lex = morph.lex.lower()
                if morph.lex in divword:
                    morph.lex = divword2[divword.index(morph.lex)]
                    list.extend(morph.lex)
                elif morph.lex in modiword:
                    morph.lex = modiword2[modiword.index(morph.lex)]
                    list.append(morph.lex)
                else: list.append(morph.lex)
                
    return list

In [31]:
tokenized = data["kor_full"].apply(lambda row: khaiiiTokenizer(row))
print(tokenized)

print("========= tokenization completed =========")

0                                     [멀티미디어, 원격, 교육, 연구]
1                               [교육, 하이퍼미디어, 자료, 편집기, 연구]
2                                     [인터넷, 코스웨어, 설계, 구현]
3                               [web, 협력, 환경, 구축, 방안, 연구]
4                                       [교육, 개별, 수업, cai]
                              ...                        
1144    [개정, 교과서, 소프트웨어, 교육, 단원, 탐구, 비교, 분석, 교육, 과정, 교...
1145    [이러닝, 콘텐츠, 사용자, 경험, ux, 평가, 이러닝, 대리, 상호, 작용, 사...
1146    [초등, 데이터, 리터러시, 함양, ai, 데이터, 과학, 교육, 프로그램, 개발,...
1147    [초등, 예비, 교사, 소프트웨어, 교육, 온라인, 교육, 효과, 분석, 소프트웨어...
1148    [초등, 교과서, 소프트웨어교육, 영역, 컴퓨팅, 사고력, 요소, 분석, 소프트웨어...
Name: kor_full, Length: 1149, dtype: object
========= tokenization completed =========


In [43]:
tokenized2 = data['kor_full'].apply(lambda row: khaiiiTokenizer(row, pos=['NNG', 'NNP', 'NNB', 'NP', 'NR', 'SL', 'VV', 'VA','MM']))
print(tokenized2)
print("========= tokenization completed =========")

0                                 [멀티미디어, 원격, 교육, 관하, 연구]
1                           [교육, 하이퍼미디어, 자료, 편집기, 관하, 연구]
2                                     [인터넷, 코스웨어, 설계, 구현]
3                               [web, 협력, 환경, 구축, 방안, 연구]
4                                   [열리, 교육, 개별, 수업, cai]
                              ...                        
1144    [개정, 교과서, 소프트웨어, 교육, 단원, 탐구, 비교, 분석, 교육, 과정, 교...
1145    [이러닝, 콘텐츠, 관하, 사용자, 경험, ux, 평가, 이러닝, 대리, 상호, 작...
1146    [초등, 데이터, 리터러시, 함양, 위하, ai, 데이터, 과학, 교육, 프로그램,...
1147    [초등, 예비, 교사, 위하, 소프트웨어, 교육, 대하, 온라인, 교육, 효과, 분...
1148    [초등, 교과서, 소프트웨어교육, 영역, 나타나, 컴퓨팅, 사고력, 요소, 분석, ...
Name: kor_full, Length: 1149, dtype: object
========= tokenization completed =========


In [44]:
# n-gram candidates

def get_ngrams(raw, n_range=(1,3)): # 1~n-gram 까지

    def to_ngrams(words, n):
        ngrams = []
        for b in range(0, len(words) - n + 1):
            ngrams.append(tuple(words[b:b+n]))
        return ngrams

    n_begin, n_end = n_range
    ngram_list = []
    
    for n in range(n_begin, n_end + 1):
        for ngram in to_ngrams(raw, n):
            ngram_list.append(ngram)
            
    return ngram_list


In [45]:
ngrams = tokenized2.apply(lambda row: get_ngrams(row))

print(ngrams)

#ngrams.to_csv('./modi_data/ngrams.csv')

0       [(멀티미디어,), (원격,), (교육,), (관하,), (연구,), (멀티미디어,...
1       [(교육,), (하이퍼미디어,), (자료,), (편집기,), (관하,), (연구,)...
2       [(인터넷,), (코스웨어,), (설계,), (구현,), (인터넷, 코스웨어), (...
3       [(web,), (협력,), (환경,), (구축,), (방안,), (연구,), (w...
4       [(열리,), (교육,), (개별,), (수업,), (cai,), (열리, 교육),...
                              ...                        
1144    [(개정,), (교과서,), (소프트웨어,), (교육,), (단원,), (탐구,),...
1145    [(이러닝,), (콘텐츠,), (관하,), (사용자,), (경험,), (ux,), ...
1146    [(초등,), (데이터,), (리터러시,), (함양,), (위하,), (ai,), ...
1147    [(초등,), (예비,), (교사,), (위하,), (소프트웨어,), (교육,), ...
1148    [(초등,), (교과서,), (소프트웨어교육,), (영역,), (나타나,), (컴퓨...
Name: kor_full, Length: 1149, dtype: object


In [47]:
ngram_list = []

for i in range(len(ngrams)):
    #print(ngrams[i])
    ngram_list.extend(ngrams[i])
#print(ngram_list)

In [48]:
from collections import defaultdict

def get_ngram_counter(ngrams):
    ngram_counter = defaultdict(int)

    for ngram in ngrams:
        ngram_counter[ngram] += 1

    return ngram_counter

In [49]:
ngram_counter = get_ngram_counter(ngram_list)

print(ngram_counter)

, ('단어', '이용', '의미'): 1, ('이용', '의미', '분별법'): 1, ('분별법', '이용', '사전'): 1, ('이용', '사전', '사후'): 1, ('사전', '사후', '인공지능'): 1, ('사후', '인공지능', '대하'): 1, ('인공지능', '대하', '학생'): 1, ('대하', '학생', '이미지'): 1, ('학생', '이미지', '인식'): 1, ('이미지', '인식', '변화'): 1, ('인식', '변화', '살펴보'): 1, ('변화', '살펴보', '학생'): 1, ('살펴보', '학생', '인공지능'): 1, ('학생', '인공지능', '대하'): 1, ('대하', '이미지', '단어'): 1, ('이미지', '단어', '긍정'): 1, ('단어', '긍정', '변화'): 1, ('아두이노',): 1, ('해결력', '컴퓨팅'): 3, ('사고', '미치'): 2, ('인공지능', '소프트웨어'): 1, ('지역', '고등학생'): 1, ('대상', '아두'): 1, ('아두', '인공지능'): 1, ('인공지능', '메이커'): 1, ('영향', '관찰'): 1, ('관찰', '위하'): 1, ('위하', 'mblock'): 1, ('mblock', '아두이노'): 1, ('아두이노', '인공지능'): 1, ('학교', '외부'): 1, ('외부', '메이커'): 1, ('스페이스', '교육'): 1, ('학생', '데이터'): 2, ('분석', '대응'): 1, ('모든', '부분'): 1, ('대하', '예측변인'): 1, ('예측변인', '확인'): 2, ('위하', '다중'): 1, ('유의', '예측변인'): 1, ('미치', '사실'): 1, ('의의', '시사점'): 1, ('연구', '제한점'): 1, ('제한점', '추후'): 1, ('추후', '연구'): 1, ('고등학생', '대상', '메이커'): 1, ('대상', '메이커', '교육'): 1, ('문제', '해결력', '컴퓨팅'): 3

In [50]:
# unigram_counter list
unigram_list = []

for i in range(len(tokenized)):
    unigram_list.extend(tokenized[i])
#print(unigram_list)

unigram_counter = get_ngram_counter(unigram_list)
unigram_counter = pd.DataFrame(unigram_counter, index=[0])
print(unigram_counter)
unigram_counter.to_excel('./modi_data/unigram_counter.xlsx')

   멀티미디어  원격    교육    연구  하이퍼미디어   자료  편집기  인터넷  코스웨어   설계  ...  대리  뉴노멀  new  \
0     59  99  4961  2115      11  346    3  280    86  583  ...   1    1    1   

   normal  독백  파급력  바이러스  출판사  교과서별  발견과  
0       1   3    1     1    1     1    1  

[1 rows x 5086 columns]


In [51]:
def get_ngram_score(ngram_counter, delta=10): # PMI 이용
    nc = pd.DataFrame()
    
    for ngram, count in ngram_counter.items():
        if len(ngram) == 1: continue

        first = ngram_counter[ngram[:-1]]
        second = ngram_counter[ngram[1:]]
        score = (count - delta) / (first * second)

        if score > 0 :
            list = [ngram, round(score * 100, 5), count]
            nc = nc.append(pd.Series(list), ignore_index = True)

    return nc

In [26]:
ngram_candidates = get_ngram_score(ngram_counter)
ngram_candidates.columns = ['후보명', 'PMI', 'count']

ngram_candidates = ngram_candidates.sort_values(by=['PMI'], axis=0, ascending=False)
ngram_candidates[:100].to_excel('./modi_data/ngram_candidates.xlsx')

print(ngram_candidates)
